In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun May 24 13:05:53 2020

@author: mahsa
"""

import tensorflow as tf
def trainDatagen():
    for npy in train_list:
        x = tf.convert_to_tensor(np.load(npy), dtype=tf.float32)
        if npy in gbmlist:
             y = to_categorical(0, num_classes=2)
        else:
             y = to_categorical(1, num_classes=2)
        yield x, y

def tfDatasetGen(datagen, output_types, is_training, batch_size):
    dataset = tf.data.Dataset.from_generator(generator=datagen, output_types=output_types)
    if is_training:
        dataset.shuffle(buffer_size=100)
    dataset.repeat()
    dataset.batch(batch_size=batch_size)
    dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

train_set = tfDatasetGen(
    datagen = trainDatagen, 
    output_types = (tf.float32, tf.float32), 
    is_training = True, 
    batch_size = 16)

def top_k(inputs, k):
    return tf.nn.top_k(inputs, k=k, sorted=True).values

def least_k(inputs, k):
    return -tf.nn.top_k(-inputs, k=k, sorted=True).values

def minmax_k(inputs, k):
    return tf.concat([top_k(inputs, k), least_k(inputs, k)], axis = -1)

inputs = keras.Input(shape=(4000,2048))
y = layers.Conv1D(1, 4000, use_bias=False, padding="same", data_format='channels_last')(inputs)
y = layers.Flatten()(y)
y = layers.Lambda(minmax_k, arguments={'k': 5})(y)
y = layers.Dense(units=200, activation=tf.nn.relu)(y)
y = layers.Dropout(rate=0.5)(y)
y = layers.Dense(units=100, activation=tf.nn.relu)(y)
y = layers.Dense(units=2, activation=tf.nn.softmax)(y)
model = keras.Model(inputs=inputs, outputs=y)